In [16]:
import pandas as pd
import csv

In [17]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [18]:
!psql -c"CREATE DATABASE bankcalls;"

ERROR:  database "bankcalls" already exists


In [19]:
%sql postgresql://localhost/bankcalls

'Connected: @bankcalls'

In [20]:
%%sql
CREATE TABLE bank_addl (
    bank_addl_id SERIAL UNIQUE,
    age INT,
    job TEXT,
    marital TEXT,
    education TEXT,
    in_default TEXT,
    housing TEXT,
    loan TEXT,
    contact TEXT,
    month TEXT,
    day_of_week TEXT,
    duration INT,
    campaign INT,
    pdays INT,
    previous INT,
    poutcome TEXT,
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    cons_conf_idx FLOAT,
    euribor3m FLOAT,
    nr_employed FLOAT,
    y TEXT    
);

 * postgresql://localhost/bankcalls
(psycopg2.ProgrammingError) relation "bank_addl" already exists
 [SQL: 'CREATE TABLE bank_addl (\n    bank_addl_id SERIAL UNIQUE,\n    age INT,\n    job TEXT,\n    marital TEXT,\n    education TEXT,\n    in_default TEXT,\n    housing TEXT,\n    loan TEXT,\n    contact TEXT,\n    month TEXT,\n    day_of_week TEXT,\n    duration INT,\n    campaign INT,\n    pdays INT,\n    previous INT,\n    poutcome TEXT,\n    emp_var_rate FLOAT,\n    cons_price_idx FLOAT,\n    cons_conf_idx FLOAT,\n    euribor3m FLOAT,\n    nr_employed FLOAT,\n    y TEXT    \n);'] (Background on this error at: http://sqlalche.me/e/f405)


In [22]:
dirname = "../data/bank-additional/"
filename = "bank-additional-full.csv"

In [23]:
# Rename the columns from the csv file 
# because default is a sql keyword, rename the loan in default column to "in_default"
# because the period is also sql syntax, rename all periods to underscores
def fix_names(csv_names):
    postgres_names = []
    for name in csv_names:
        if name == "default":
            postgres_names += ["in_default"]
        else:
            postgres_names += [name.replace('.','_')]
    return postgres_names
        

In [24]:
# Explicitly adds a single quote around strings that SQL is supposed to interpret
# as strings
def quote_strings(row_df):
    categorical_columns = [ 'job', 'marital', 'education', 'in_default', 'housing',
                         'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y' ]
    row_df.loc[categorical_columns] = "'" + row_df.loc[categorical_columns] + "'"
    
    

In [26]:
# 41188 postgresql feedback strings is not pretty 
# The %%capture in the next cell captures even more output
%config SqlMagic.feedback   = False
%config SqlMagic.autocommit = False

In [27]:
%%capture
with open(dirname + filename, newline='') as csvfile:

    # figure out how to read the csv file
    dialect = csv.Sniffer().sniff(csvfile.read(1024), delimiters=';')
    csvfile.seek(0)
    reader = csv.reader(csvfile, dialect)

    # pull off the header and turn it into my postgres column names
    header = next(reader)
    postgres_names = fix_names(header)
    postgres_columns = ", ".join(postgres_names)

    for row in reader:
        row_df = pd.DataFrame(index=postgres_names, data=row)
        quote_strings(row_df)
        row_values = ", ".join(row_df.loc[:, 0])
        command = ("INSERT INTO bank_addl ( " + postgres_columns + ") " +
                   " VALUES (" + row_values + " );")
        result = %sql $command
        if result == None:
            raise Exception('SQL command failed>', command)


In [28]:
%config SqlMagic.feedback = True
%config SqlMagic.autocommit = True


In [29]:
%sql commit; select count(*) from bank_addl;

 * postgresql://localhost/bankcalls
Done.
1 rows affected.


count
41188
